<a href="https://colab.research.google.com/github/durfred/my-first-binder/blob/main/CV_Evaluator_(RAG_%2B_llama_cpp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets sentence-transformers faiss-cpu llama-cpp-python --quiet


In [20]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class RAGEngine:
    def __init__(self, docs):
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.docs = docs
        self.embeddings = self.embedder.encode(docs)

    def retrieve(self, query, top_k=3):
        q_embed = self.embedder.encode([query])
        scores = cosine_similarity(q_embed, self.embeddings)[0]
        top_idxs = np.argsort(scores)[-top_k:][::-1]
        return [self.docs[i] for i in top_idxs]


In [ ]:
!wget https://huggingface.co/codegood/gemma-2b-it-Q4_K_M-GGUF/resolve/main/gemma-2b-it.Q4_K_M.gguf

In [ ]:
from llama_cpp import Llama

llm = Llama(model_path="gemma-2b-it.Q4_K_M.gguf", n_ctx=2048)

def query_model(prompt):
    response = llm(prompt, max_tokens=256)
    return response["choices"][0]["text"]



In [25]:
from sentence_transformers import SentenceTransformer
import faiss

# Embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Simulated knowledge base (can replace with UK job guide text)
knowledge_docs = [
    "Civil Service Level 3 requires experience in policy development, written communication, and working with stakeholders.",
    "Software Engineer should know Python, Git, and agile development practices.",
    "Project Managers are expected to manage budgets, timelines, and interdepartmental coordination.",
    "Data analysts should demonstrate SQL, Excel, and experience with government datasets."
]

# Build FAISS index
doc_embeddings = embedder.encode(knowledge_docs)
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)


In [26]:
import numpy as np

def retrieve_context(query, k=3):
    query_embed = embedder.encode([query])
    _, indices = index.search(np.array(query_embed), k)
    return "\n".join([knowledge_docs[i] for i in indices[0]])

def build_prompt(jd, cv, context):
    return f"""
You are a UK government HR specialist.

--- Context Knowledge ---
{context}

--- Job Description ---
{jd}

--- Candidate CV ---
{cv}

Evaluate the CV against the job:
1. Estimate match percentage
2. Matching skills or points
3. Missing or weak points
4. Suggestions to improve the CV

Return the result in JSON format.
"""

def evaluate_cv(jd, cv):
    context = retrieve_context(jd + "\n" + cv)
    prompt = build_prompt(jd, cv, context)
    response = llm(prompt, max_tokens=512)
    return response["choices"][0]["text"]


In [ ]:
import gradio as gr

iface = gr.Interface(
    fn=evaluate_cv,
    inputs=[
        gr.Textbox(label="Job Description", lines=6),
        gr.Textbox(label="Candidate CV", lines=12)
    ],
    outputs="text",
    title="📄 CV Evaluator (RAG + llama.cpp)",
    description="Evaluates how well a CV matches a job description using local Mistral + vector RAG."
)

iface.launch(share=True)
